In [2]:

import cv2
import numpy as np

background = cv2.imread('building.jpg')   # architectural image
overlay = cv2.imread('flag.png')          # flag image

# Resize overlay to a manageable size
overlay = cv2.resize(overlay, (300, 200))

points = []
def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))
        print(f"Point selected: {x}, {y}")

img_copy = background.copy()
cv2.imshow('Select 4 points', img_copy)
cv2.setMouseCallback('Select 4 points', mouse_callback)
cv2.waitKey(0)
cv2.destroyAllWindows()
print("Points:", points)

dst_pts = np.array(points, dtype=np.float32)


# source points (corners of the flag)
h, w = overlay.shape[:2]
src_pts = np.array([
    [0, 0],
    [w - 1, 0],
    [w - 1, h - 1],
    [0, h - 1]
], dtype=np.float32)

# Compute Homography and warp the overlay image
H, _ = cv2.findHomography(src_pts, dst_pts)
warped_flag = cv2.warpPerspective(overlay, H, (background.shape[1], background.shape[0]))

# Binary mask of the quad
mask = np.zeros(background.shape[:2], dtype=np.uint8)
cv2.fillConvexPoly(mask, dst_pts.astype(int), 255)

# Convert to float for proper blending
bg_f   = background.astype(np.float32)
flag_f = warped_flag.astype(np.float32)
alp    = 0.5  # 0=transparent, 1=opaque

# Build per-pixel alpha in [0,1] but only inside the quad
a = (mask.astype(np.float32) / 255.0) * alp

# Broadcast alpha to 3 channels
a3 = cv2.merge([a, a, a])

# Blend INSIDE the mask with original background pixels
result_f = bg_f * (1.0 - a3) + flag_f * a3

# Keep background outside the mask exactly unchanged
result = result_f.astype(np.uint8)

# --- tiny border around the flag quad (AFTER blending) ---
# choose a visible color (BGR): yellow, cyan, white, etc.
border_color = (0, 255, 255)   # yellow
thickness    = 2               # 1–3 px looks good
dst_poly     = dst_pts.astype(np.int32).reshape(-1, 1, 2)

# draw on the final result so it stays crisp (not affected by alpha)
cv2.polylines(result, [dst_poly], isClosed=True, color=border_color,
              thickness=thickness, lineType=cv2.LINE_AA)


cv2.imshow('Transparent Flag Overlay', result)
cv2.imwrite('overlay_transparent_result.jpg', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

Point selected: 503, 278
Point selected: 864, 130
Point selected: 860, 396
Point selected: 504, 448
Points: [(503, 278), (864, 130), (860, 396), (504, 448)]
